In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, optimizers, Sequential

def load_dataset1(a):
    (x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=a)
    return y_train

y = load_dataset1(10000)
print(y[0:20])
    
#     x_train = keras.preprocessing.sequence.pad_sequences(x_train, b)
#     x_test = keras.preprocessing.sequence.pad_sequences(x_test, b)
#     """
#     keras只能够接收等长度的序列输入，因此.preprocessing.sequence.pad_sequences是将一个输入序列转化为一个长度相同的序列输入。
#     keras.preprocessing.sequence.pad_sequences(sequences, maxlen=None,dtype='int32',padding='pre',truncating='pre', value=0.)
#     sequences：浮点数或整数构成的两层嵌套列表
#     maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
#     dtype：返回的numpy array的数据类型
#     padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补`
#     truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
#     value：浮点数，此值将在填充时代替默认的填充值0
#     """
#     db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#     db_train = db_train.shuffle(1000).batch(batchsz, drop_remainder=True)   ####当最后一个batch不够batchsize的时候，使用drop_remainder来确定是否丢弃。
#     db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
#     db_test = db_test.batch(batchsz, drop_remainder=True)
#     return db_train, db_test

def load_dataset(batchsz, total_words, max_review_len):
    # 加载IMDB数据集，此处的数据采用数字编码，一个数字代表一个单词
    (x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=total_words)
    print(x_train.shape, len(x_train[0]), y_train.shape)
    print(x_test.shape, len(x_test[0]), y_test.shape)

    # x_train:[b, 80]
    # x_test: [b, 80]
    # 截断和填充句子，使得等长，此处长句子保留句子后面的部分，短句子在前面填充
    x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
    x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)
    # 构建数据集，打散，批量，并丢掉最后一个不够batchsz的batch
    db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    db_train = db_train.shuffle(1000).batch(batchsz, drop_remainder=True)
    db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    db_test = db_test.batch(batchsz, drop_remainder=True)
    print('x_train shape:', x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
    print('x_test shape:', x_test.shape)
    return db_train, db_test


D:\Anaconda3\envs\tensorflow2.x\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


[1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 1]


D:\Anaconda3\envs\tensorflow2.x\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [2]:
class RNNModel(keras.Model):
    def __init__(self, units, batchsz, total_words, embedding_len, max_review_len):
        super(RNNModel, self).__init__()
        # [b, 64]，构建Cell初始化状态向量，重复使用
        self.state0 = [tf.zeros([batchsz, units])]
        self.state1 = [tf.zeros([batchsz, units])]
        # 词向量编码 [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len, input_length=max_review_len)
        # 构建2个Cell
        self.rnn_cell0 = layers.SimpleRNNCell(units, dropout=0.5)
        self.rnn_cell1 = layers.SimpleRNNCell(units, dropout=0.5)
        # 构建分类网络，用于将CELL的输出特征进行分类，2分类
        # [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = Sequential([
            layers.Dense(units),
            layers.Dropout(rate=0.5),
            layers.ReLU(),
            layers.Dense(1)])

    def call(self, inputs, training=None):
        x = inputs  # [b, 80]
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute,[b, 80, 100] => [b, 64]
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1):  # word: [b, 100]
            out0, state0 = self.rnn_cell0(word, state0, training)
            out1, state1 = self.rnn_cell1(out0, state1, training)
        # 末层最后一个输出作为分类网络的输入: [b, 64] => [b, 1]
        x = self.outlayer(out1, training)
        # p(y is pos|x)
        prob = tf.sigmoid(x)

        return prob

In [3]:
def main():
    batchsz = 100  # 批量大小
    total_words = 10000  # 词汇表大小N_vocab
    embedding_len = 100  # 词向量特征长度f
    max_review_len = 80  # 句子最大长度s，大于的句子部分将截断，小于的将填充

    db_train, db_test = load_dataset(batchsz, total_words, max_review_len)

    units = 64  # RNN状态向量长度f
    epochs = 20  # 训练epochs

    model = RNNModel(units, batchsz, total_words, embedding_len, max_review_len)
    # 装配
    model.compile(optimizer=optimizers.RMSprop(0.001),
                  loss=losses.BinaryCrossentropy(),
                  metrics=['accuracy'],
                  experimental_run_tf_function=False)
    # 训练和验证
    model.fit(db_train, epochs=epochs, validation_data=db_test)
    # 测试
    model.evaluate(db_test)


In [ ]:
if __name__ == '__main__':
    main()

(25000,) 218 (25000,)
(25000,) 68 (25000,)
